# Testing Notebook

Load a model and test it against images of your choice.

ipywidgets reference: https://medium.com/data-science/interactive-controls-for-jupyter-notebooks-f5c94829aee6

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Loading Model

In [ ]:
# Loading Model
import os
from setup import SAVE_PATH, FINE_TUNE
from torchvision.models import resnet18
from ipywidgets import interact
from model.load_model import load_saved_model

data = {'model': None}

@interact
def choose_model(selected_model=os.listdir(SAVE_PATH), fine_tune=False):
    model_path = os.path.join(SAVE_PATH, selected_model)
    model = load_saved_model(model_path, fine_tune=fine_tune)

    model.eval()
    model.to(device)
    data['model'] = model
    print('Model loaded successfully')

## Testing On Specific Image

In [ ]:

from IPython.display import Image
from setup import DATA_PATH, IMAGE_HEIGHT, IMAGE_WIDTH
from torchvision import transforms
from model.run_img import run_img

@interact
def set_folder(class_name='Covid', subset='test'):
    PATH = os.path.join(DATA_PATH, subset, class_name)
    
    @interact
    def choose_img(img_name=os.listdir(os.path.join(PATH))):

        # Img
        img_path = os.path.join(PATH, img_name)
        transform = transforms.Compose([transforms.Resize(size=(IMAGE_WIDTH, IMAGE_HEIGHT)), transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

        predicted_class, _ = run_img(data['model'], img_path, device, transform)

        # Display Results
        display(f'Predicted Class: {predicted_class}')
        display(f'Class: {class_name}')
        display(Image(os.path.join(PATH, img_name)))

## Evaluating Image By Image

In [ ]:
from data.evaluation import evaluate_single

evaluate_single(model=data['model'], classes_list=['Covid', 'Normal', 'Viral Pneumonia'], device=device)